# Importing libraries

In [1]:
import numpy as np
import pandas as pd
import torch
from datetime import datetime

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, pipeline
from transformers import BertForQuestionAnswering, BertTokenizer
from transformers import AutoModelForCausalLM, AutoModel
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama

# Getting from csv and splitting

In [3]:
csv_file_path = 'example_data.csv'

loader = CSVLoader(file_path=csv_file_path)

data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

docs = text_splitter.split_documents(data)

In [5]:
docs[0]

Document(metadata={'source': 'example_data.csv', 'row': 0}, page_content=': \ncategory: Eğitim 101\nmain_title: Ders Seçimi\nsub_title: main text\ncontent: ODTÜ 2022-2023 yılı sonbahar dönemi için etkileşimli kayıt sistemi 28 Eylül 2022 tarihinde, saat 09.00’da öğrencilere açılacak. Birer saat arayla, başta 4. sınıflar olmak üzere tüm sınıflara kademeli olarak açılmaya devam edecek. Etkileşimli kayıtlar 30 Eylül, saat 17.00’de sona erecektir. Danışman onayları ise 28 Eylül’de başlayıp 30 Eylül saat 23.59’da sona erecektir. Not: Kayıt programına kampüs dışından erişimlerde VPN bağlantısı şartı bulunmamaktadır.')

# Getting an pre-trained embedding model

In [6]:
model_path = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceEmbeddings(
    model_name = model_path,      
    model_kwargs = model_kwargs,  
    encode_kwargs = encode_kwargs 
)

/tmp/ipykernel_120188/1696297337.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [7]:
text = 'Üreten ekibe hoş geldin'
query_result = embeddings.embed_query(text)
len(query_result)

384

# Creating a vector database

In [8]:
vector_db = FAISS.from_documents(docs, embeddings)

In [9]:
question = "metubot kim tarafından geliştirilmektedir"
searchDocs = vector_db.similarity_search(question)
print(searchDocs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


# Creating a retriever

In [10]:
retriever = vector_db.as_retriever()

docs = retriever.invoke("metubot kim tarafından geliştirilmektedir")
print(docs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


# Getting a generative model and creating a retrieval qa chain

In [11]:
llm = ChatOllama(
    model="llama3.2:3b",
    temperature=0.1,
)

In [12]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

# Testing the RAG model

In [13]:
question = "metubot kim tarafından geliştirilmektedir"

In [14]:
docs = retriever.invoke(question)
print(docs[0])

page_content=': 
category: Deneme
main_title: Topluluklar
sub_title: Topluluklar
content: metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.' metadata={'source': 'example_data.csv', 'row': 102}


In [15]:
start_time = datetime.now()

result = qa.invoke(question)
print(result['result'])

end_time = datetime.now()

Metubot projesi GDGC METU AI/ML Takımı Üreten Ekibi tarafından geliştirilmektedir.


In [16]:
elapsed_time = end_time - start_time
print(f"Geçen Süre: {elapsed_time}")

Geçen Süre: 0:00:00.502124


# Conclusion

**Models used:**

Pre-trained embedding model:
* sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
* https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2

Generative model:
* ollama llama3.2 3b